In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('/content/diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [6]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x=scaler.fit_transform(x)

In [8]:
x.shape

(768, 8)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [62]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [12]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7770 - loss: 0.4698 - val_accuracy: 0.8182 - val_loss: 0.4647
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8008 - loss: 0.4516 - val_accuracy: 0.8182 - val_loss: 0.4644
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7781 - loss: 0.4610 - val_accuracy: 0.8117 - val_loss: 0.4641
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7908 - loss: 0.4417 - val_accuracy: 0.8117 - val_loss: 0.4636
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8134 - loss: 0.4412 - val_accuracy: 0.8052 - val_loss: 0.4638
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7900 - loss: 0.4658 - val_accuracy: 0.8052 - val_loss: 0.4657
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8013 - loss: 0.4368 - val_accuracy: 0.7987 - val_loss: 0.4646
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7892 - loss: 0.4497 - val_accuracy: 0.8117 - v

In [16]:
# how to select appropriate optimizer
# no. of nodes in a layer
# how to select no. of layers
#all in all one model

In [18]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.0 MB/s eta 0:00:00


In [19]:
import kerastuner as kt

In [21]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))


  model.compile(optimizer=hp.Choice('optimizer',['adam','sgd','rmsprop','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [22]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [23]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.4545454680919647

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 00m 12s


In [24]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [26]:
model=tuner.get_best_models(num_models=1)[0]

In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.fit(x_train,y_train,batch_size=32,initial_epoch=6,  epochs=100,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7445 - loss: 0.5158 - val_accuracy: 0.7662 - val_loss: 0.5084
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7176 - loss: 0.5236 - val_accuracy: 0.7662 - val_loss: 0.4966
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7306 - loss: 0.5244 - val_accuracy: 0.7662 - val_loss: 0.4883
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7574 - loss: 0.5117 - val_accuracy: 0.7792 - val_loss: 0.4812
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7445 - loss: 0.5013 - val_accuracy: 0.7922 - val_loss: 0.4761
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7884 - loss: 0.4494 - val_accuracy: 0.7922 - val_loss: 0.4732
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7730 - loss: 0.4765 - val_accuracy: 0.7792 - val_loss: 0.4703
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7912 - loss: 0.4605 - val_accuracy: 0.77

In [32]:
def build_model(hp):
  model=Sequential()
  units=hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [33]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir')

In [34]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 15s


In [35]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [38]:
model=tuner.get_best_models(num_models=1)[0]

In [40]:
model.fit(x_train,y_train,batch_size=32,initial_epoch=6,  epochs=100,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7781 - loss: 0.4715 - val_accuracy: 0.7922 - val_loss: 0.4720
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7818 - loss: 0.4701 - val_accuracy: 0.7857 - val_loss: 0.4692
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7723 - loss: 0.4682 - val_accuracy: 0.7987 - val_loss: 0.4678
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7640 - loss: 0.4687 - val_accuracy: 0.7987 - val_loss: 0.4665
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8034 - loss: 0.4356 - val_accuracy: 0.7922 - val_loss: 0.4662
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7572 - loss: 0.4696 - val_accuracy: 0.7857 - val_loss: 0.4655
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7784 - loss: 0.4698 - val_accuracy: 0.7792 - val_loss: 0.4652
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7802 - loss: 0.4500 - val_accuracy: 0.78

In [41]:
def build_model(hp):
  model=Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
   model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [42]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='num_layers')

In [43]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 10s


In [44]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [45]:
model=tuner.get_best_models(num_models=1)[0]

In [46]:
model.fit(x_train,y_train,batch_size=32,initial_epoch=6,  epochs=100,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7693 - loss: 0.4609 - val_accuracy: 0.7922 - val_loss: 0.4825
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7869 - loss: 0.4253 - val_accuracy: 0.7792 - val_loss: 0.4902
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7953 - loss: 0.4161 - val_accuracy: 0.7727 - val_loss: 0.5066
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7889 - loss: 0.4107 - val_accuracy: 0.7987 - val_loss: 0.4942
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8076 - loss: 0.3858 - val_accuracy: 0.7922 - val_loss: 0.4877
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7904 - loss: 0.4223 - val_accuracy: 0.7792 - val_loss: 0.5484
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8115 - loss: 0.3802 - val_accuracy: 0.7727 - val_loss: 0.5155
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8155 - loss: 0.3749 - val_accuracy: 0.76

In [63]:
def build_model(hp):
  model=Sequential()

  counter=0


  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter ==0 :
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:
       model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])))
       model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adamdelta']),loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [64]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='final1')

In [65]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 13s


In [66]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 80,
 'activation0': 'relu',
 'dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 128,
 'activation1': 'relu',
 'dropout1': 0.4,
 'units2': 80,
 'activation2': 'relu',
 'dropout2': 0.6,
 'units3': 88,
 'activation3': 'tanh',
 'dropout3': 0.6,
 'units4': 40,
 'activation4': 'sigmoid',
 'dropout4': 0.2,
 'units5': 16,
 'activation5': 'tanh',
 'dropout5': 0.8,
 'units6': 16,
 'activation6': 'tanh',
 'dropout6': 0.7,
 'units7': 64,
 'activation7': 'tanh',
 'dropout7': 0.5,
 'units8': 80,
 'activation8': 'sigmoid',
 'dropout8': 0.4,
 'units9': 88,
 'activation9': 'sigmoid',
 'dropout9': 0.8}

In [67]:
model=tuner.get_best_models(num_models=1)[0]

In [68]:
model.fit(x_train,y_train,batch_size=32,initial_epoch=6,  epochs=200,validation_data=(x_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7781 - loss: 0.5169 - val_accuracy: 0.7987 - val_loss: 0.4889
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7706 - loss: 0.4815 - val_accuracy: 0.7922 - val_loss: 0.4758
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8051 - loss: 0.4781 - val_accuracy: 0.7922 - val_loss: 0.4673
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7744 - loss: 0.4777 - val_accuracy: 0.7922 - val_loss: 0.4644
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7599 - loss: 0.4674 - val_accuracy: 0.7922 - val_loss: 0.4602
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7479 - loss: 0.5056 - val_accuracy: 0.7857 - val_loss: 0.4591
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7699 - loss: 0.4597 - val_accuracy: 0.7727 - val_loss: 0.4564
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7655 - loss: 0.4505 - val_accuracy: 0.77